# guid

In [ ]:
open sm'_operators

In [ ]:
//// test

open testing

## guid

### guid

In [ ]:
nominal guid_python =
    `(
        global "import uuid"
        $'' : $'uuid.UUID'
    )
type guid_switch =
    {
        Gleam : string
        Fsharp : $'System.Guid'
        Python : guid_python
    }
nominal guid = $'backend_switch `(guid_switch)'

### new_guid

In [ ]:
inl new_guid (x : string) : guid =
    run_target_args (fun () => x) function
        | Rust (Contract) => fun _ => null ()
        | _ => fun x => x |> convert

### new_raw_guid

In [ ]:
inl new_raw_guid () : guid =
    backend_switch {
        Gleam = fun () =>
            global "import gleam_uuid"
            $'gleam_uuid.v4_string ()' : guid
        Fsharp = fun () => $'System.Guid.NewGuid' () : guid
        Python = fun () => $'uuid.uuid4()' : guid
    }

### hash_guid

In [ ]:
type hash_guid = guid

let hash_guid (hash : string) : hash_guid =
    inl hash = hash |> sm'.pad_left 32i32 '0'
    run_target_args (fun () => "-", hash) function
        | Rust (Contract) => fun _ => null ()
        | _ => fun dash, hash =>
            inl a = hash |> sm'.range (am'.Start 0i32) (am'.End fun _ => 8)
            inl b = hash |> sm'.range (am'.Start 8i32) (am'.End fun _ => 12)
            inl c = hash |> sm'.range (am'.Start 12i32) (am'.End fun _ => 16)
            inl d = hash |> sm'.range (am'.Start 16i32) (am'.End fun _ => 20)
            inl e = hash |> sm'.range (am'.Start 20i32) (am'.End fun _ => 32)
            a ++# dash ++# b ++# dash ++# c ++# dash ++# d ++# dash ++# e
            |> new_guid

In [ ]:
//// test
///! gleam
///! fsharp
///! cuda
///! rust
///! typescript
///! python

""
|> hash_guid
|> _assert_eq' (
    backend_switch {
        Gleam = fun () => new_guid "00000000-000000000000-0000000000000000-0000000000000000-000000000000"
        Fsharp = fun () => new_guid "00000000-0000-0000-0000-000000000000"
        Python = fun () => new_guid "00000000-0000-0000-0000-000000000000"
    }
)

"123456789012345678901234567890123"
|> hash_guid
|> _assert_eq' (
    backend_switch {
        Gleam = fun () => new_guid "12345678-901234567890-3456789012345678-78901234567890123-1234567890123"
        Fsharp = fun () => new_guid "12345678-9012-3456-7890-123456789012"
        Python = fun () => new_guid "12345678-9012-3456-7890-123456789012"
    }
)


.py output (Python):
{ name = __assert_eq'; expected = 00000000-0000-0000-0000-000000000000 }
{ name = __assert_eq'; expected = 12345678-9012-3456-7890-123456789012 }


.rs output:
{ name = __assert_eq'; expected = Guid(
    00000000-0000-0000-0000-000000000000,
) }
{ name = __assert_eq'; expected = Guid(
    12345678-9012-3456-7890-123456789012,
) }


.ts output:
{ name = __assert_eq'; expected = 00000000-0000-0000-0000-000000000000 }
{ name = __assert_eq'; expected = 12345678-9012-3456-7890-123456789012 }


.py output:
{ name = __assert_eq'; expected = 00000000-0000-0000-0000-000000000000 }
{ name = __assert_eq'; expected = 12345678-9012-3456-7890-123456789012 }


.gleam output (Gleam):
{ name = __assert_eq'; expected = "00000000-000000000000-0000000000000000-0000000000000000-000000000000" }
{ name = __assert_eq'; expected = "12345678-901234567890-3456789012345678-78901234567890123-1234567890123" }




.fsx output:
{ name = __assert_eq'; expected = 00000000-0000-0000-0000-000000000000 }
{ name = __assert_eq'; expected = 12345678-9012-3456-7890-123456789012 }


## main

In [ ]:
inl main () =
    $'let new_guid x = !new_guid x' : ()
    $'let hash_guid x = !hash_guid x' : ()
    $'let new_raw_guid x = !new_raw_guid x' : ()